In [1]:
using Revise
using LinearAlgebra
using Tonari
using Plots
using Random
using Meiran

rng = MersenneTwister(1234);

Define the power spectral densities for the two processes and the cross-spectral density between them 

In [2]:
p1 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
p2 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
Δϕ = ConstantTimeLag(5.4)
cs = CrossSpectralDensity(p1, p2, Δϕ)

T = 120.0
Δt = 1.0
t = 0.0:Δt:T-Δt
S_low, S_high = 5, 5

f0, fM = 1 / T / S_low, 1 / (2Δt) * S_high
J = 1000

1000

In [4]:
σ_X₁², σ_X₂² = zeros(length(t)), zeros(length(t))
Σ = BlockMatrix_from_cs(cs, t, σ_X₁², σ_X₂², f0, fM, J)
sanity_checks(Σ)

In [ ]:
L₁₁, L₂₁, L₂₂ = cholesky(Σ)